In [89]:
import os, re
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import treebank
from nltk.corpus import dependency_treebank
from nltk.grammar import DependencyGrammar
from nltk.parse import DependencyGraph
import pprint
from nltk.parse.corenlp import CoreNLPDependencyParser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import numpy as np
import re

In [8]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [9]:
class Habitat:
    oid = -1
    name = ""

In [10]:
train_path = "./BioNLP-OST-2019_BB-norm_train/"
test_path = "./BioNLP-OST-2019_BB-norm_test/"
dev_path = "./BioNLP-OST-2019_BB-norm_dev/"
BB_train = os.listdir(train_path)
BB_test = os.listdir(test_path)
BB_dev = os.listdir(dev_path)
obp_file = "OntoBiotope_BioNLP-OST-2019.obo"

In [11]:
ontology = open(obp_file,encoding="utf8").read()

In [12]:
BB_train_a1 = sorted([name for name in BB_train if name.endswith("a1")])
BB_train_a2 = sorted([name for name in BB_train if name.endswith("a2")])
BB_train_txt = sorted([name for name in BB_train if name.endswith("txt")])

BB_dev_a1 = sorted([name for name in BB_dev if name.endswith("a1")])
BB_dev_a2 = sorted([name for name in BB_dev if name.endswith("a2")])
BB_dev_txt = sorted([name for name in BB_dev if name.endswith("txt")])

BB_test_a1 = sorted([name for name in BB_test if name.endswith("a1")])
BB_test_txt = sorted([name for name in BB_test if name.endswith("txt")])

In [13]:
def original_phrase(phrase):
    return phrase

In [14]:
def stem_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = ps.stem(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [15]:
def lemmatize_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = wnl.lemmatize(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [16]:
def get_headword(phrase):
    #print(phrase)
    if phrase == phrase.upper():
        return ""
    
    parses = dep_parser.parse(phrase.split())
    triples = [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses]
    
    try:
        if(len(triples) > 0 and len(triples[0]) > 0 and len(triples[0][0]) > 0 and len(triples[0][0][0]) > 0 and triples[0][0][0][0] is not None):
            return triples[0][0][0][0]
        else:
            return phrase
    except:
        return phrase
    
    

In [25]:
def add_documents(txt, processor):
    document = []
    txt = processor(txt)
    sentence_list = sent_tokenize(txt)
    
    for sentence in sentence_list:
        word_list = word_tokenize(sentence)
        word_list = [processor(word) for word in word_list if word not in stopwords.words("english") and word.isalnum()]
        for i in range(len(word_list)):
            if word_list[i].isnumeric():
                word_list[i] = "##number##"
        document.append(word_list)
    #print(document)
    return document

In [26]:
documents = []
for i in range(len(BB_train_txt)):
    txt = open(train_path + BB_train_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_dev_txt)):
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_test_txt)):
    txt = open(test_path + BB_test_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)

In [27]:
model = Word2Vec(documents, min_count=1)

In [29]:
len(model.wv.vocab)

5721

In [35]:
type(list(model.wv.vocab))

list

In [37]:
len(model[list(model.wv.vocab)[0]])

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


100

In [102]:
def get_phrase_embedding(phrase, model, processor):
    original_phrase = phrase
    phrase = phrase.replace("-", " ")
    phrase = nltk.word_tokenize(phrase)
    
    phrase = [processor(word) for word in phrase if word.lower() not in stopwords.words("english") and word.isalnum()]
    for i in range(len(phrase)):
        if phrase[i].isnumeric():
            phrase[i] = "##number##"
    
    phrase_size = len(phrase)
    vec_size = len(model[list(model.wv.vocab)[0]])
    embedded_sum = np.zeros(vec_size)
    if(phrase_size == 0):
        print(original_phrase)
        return -2
    for word in phrase:
        if word not in list(model.wv.vocab):
            return -1
        embedded_sum = embedded_sum + np.array(model[word])
    embedded_sum = embedded_sum / phrase_size
    
    return embedded_sum
    

In [96]:
phrase_reducer = lemmatize_phrase

In [97]:
phrase_reducer("related")

'related'

In [105]:
def generate_habitat_map(processor, model):
    habitat_list = ontology.split("\n\n[Term]\n")[1:]
    habitat_map = {}
    habitat_map_originals = {}
    habitat_embeddings = {}
    for h in habitat_list:
        #new_habitat = Habitat()
        h = h.split("\n")
        oid = re.findall(r"(?<=id: OBT:)[0-9]+", h[0])[0]
        name = (re.findall(r"(?<=name: ).+", h[1])[0])
        original_name = name
        if name != name.upper():
            name = name.lower()
            name = processor(name)
    
        habitat_map[name] = oid
        habitat_map_originals[oid] = name
        embedding = get_phrase_embedding(name, model, processor)
        if type(embedding) == type(np.array([1])):
            habitat_embeddings[name] = embedding
        elif embedding == -2:
            print(original_name)
        
        # word2vec buraya eklenmeli 
        for h_line in h:
            if "synonym" in h_line:
                synonym_name = (re.findall(r"(?<=synonym: \").+(?=\")", h_line)[0])
                if synonym_name != synonym_name.upper():
                    synonym_name = synonym_name.lower()
                    synonym_name = processor(synonym_name)
                if "EXACT" in h_line and synonym_name not in habitat_map:
                    habitat_map[synonym_name] = oid
                    habitat_embeddings[synonym_name] = embedding 
    return habitat_map, habitat_map_originals, habitat_embeddings

In [106]:
habitat_map, habitat_map_originals, habitat_embeddings = generate_habitat_map(phrase_reducer, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


can
can


In [14]:
def add_habitats(a1, a2, processor):
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]: 
                named_entity = (a1[a1_line].split("\t")[2])
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                if named_entity not in habitat_map:
                    habitat_map[named_entity] = oid
                    #print("Newly added entity:", named_entity, oid)
                    #print("Original entity:", habitat_map_originals[oid])

In [15]:
for i in range(len(BB_train_txt)):    
    a1 = open(train_path + BB_train_a1[i],encoding="utf8").read()
    a2 = open(train_path + BB_train_a2[i],encoding="utf8").read()
    add_habitats(a1, a2, phrase_reducer)

In [16]:
def predict_habitats_exact_matching(a1, txt, a1_name, processor, pred_file_name):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            cands += 1
            name = (line.split("\t")[2])
            named_entity = name
            if name != name.upper():
                name = name.lower()
                named_entity = processor(name)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + "---exact"
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], named_entity, "exact"]
                found = True
            else:
                #print(name)
                pre_named_entity = get_headword(name)
                #print(pre_named_entity)
                named_entity = processor(pre_named_entity)
                if named_entity in habitat_map:
                    matches += 1
                    match = name + " - " + habitat_map[named_entity] + "---headwordexact"
                    print(match)
                    match_list[line.split("\t")[0]] = [habitat_map[named_entity], name, "headwordexact", named_entity]
                    found = True
        if found:
            pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
            count += 1
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [22]:
len(documents)

1348

In [ ]:
def predict_habitats(a1, txt, a1_name, processor, pred_file_name, model):

In [ ]:
annotations_test = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    pred_file_name = "./dev_preds/" + BB_dev_a2[i]
    annotations_test[BB_dev_a1[i]] = predict_habitats(a1, txt, BB_dev_a1[i], phrase_reducer, pred_file_name)

In [ ]:
annotations_test

In [ ]:
def test_habitats(a1, a2, processor):
    
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    match_list = {}
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]:
                named_entity = (a1[a1_line].split("\t")[2])
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                match_list[t] = [oid, named_entity]
    return match_list

In [ ]:
dev_a2 = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    a2 = open(dev_path + BB_dev_a2[i],encoding="utf8").read()
    dev_a2[BB_dev_a1[i]] = test_habitats(a1,a2, phrase_reducer)

In [ ]:
true_pred_count = 0
false_pred_count = 0
total_count = 0
for key in dev_a2:
    """print()
    print(key)
    print(dev_a2[key])
    print("**************")"""
    total_count += len(dev_a2[key])
    if key in annotations_test:
        for pred in annotations_test[key]:            
            if annotations_test[key][pred][0] == dev_a2[key][pred][0]:
                true_pred_count += 1
            else:
                print(key)
                print("Prediction:", annotations_test[key][pred]," Real:", dev_a2[key][pred])
                print("Prediction:", habitat_map_originals[annotations_test[key][pred][0]], " Real:", habitat_map_originals[dev_a2[key][pred][0]])
                false_pred_count += 1

In [ ]:
recall = true_pred_count/total_count
recall

In [ ]:
precision = true_pred_count/(true_pred_count + false_pred_count)
precision

## Denemeler